<a href="https://colab.research.google.com/github/nspeer12/AI_CAP4630/blob/master/hw4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np

### Homework 4

# Problem 1

Using only ```numpy```, implement the function ```conv2d```.  It takes as input ```input_mat``` and ```kernel_mat``` and outputs ```output_mat```.  All variables 
are square matrices.  It should compute the convolution of ```input_mat``` with the kernel ```kernel_mat``` using valid padding.



## Convolutions
Convolutions are an operation that involve two input parameters. Firstly, an input matrix is passed, and secondly a kernel, or filter is defined. In convolutions, the kernel iterates through the columns and rows of the input matrix by a specified stride length. The regions considered are allowed to overlap. If the input, kernel, and stride length do not fit, the input matrix can be padded with zeros so the operation can be performed. The kernel iterates, and considers each subset of the input matrix. The dot product between the subset of the input and the kernel is performed, and then all of the values in this product are summed together. This final value becomes a value of our output matrix.

In [2]:
class invalidInput(Exception):
  print("Error: Input matrix is invalid!")
  exit(1)
  
class invalidKernel(Exception):
  print("Error: Kernel matrix is invalid!")
  exit(1)

Error: Input matrix is invalid!
Error: Kernel matrix is invalid!


In [0]:
def conv2d(input_mat, kernel_mat):
  #input_mat = np.matrix(input_mat)
  #kernel_mat = np.matrix(kernel_mat)
  # check for square inputs
  if kernel_mat.shape[0] != kernel_mat.shape[1]:
    raise invalidKernel()
  elif input_mat.shape[0] != input_mat.shape[1]:
    raise invalidInput()
  else:
    # get the dimensions of the output layer
    n = input_mat.shape[0] - kernel_mat.shape[0] + 1
    m = kernel_mat.shape[0]
  

  output_mat = np.zeros((n, n))

  for i in range(n):
    for k in range(n):
      # multiply by kernel filter
      #print(np.multiply(input_mat[i:i+m, k:k+m], kernel_mat))
      #print()
      conv = np.multiply(input_mat[i:i+m, k:k+m], kernel_mat)
      # sum all elements
      output_mat[i,k] = np.sum(conv)

  return output_mat

# Test Cases

In [0]:
def check_conv2d(input_mat, kernel_mat, expected_mat):
  res = conv2d(input_mat, kernel_mat)
  if (res == expected_mat).all():
    return True
  else:
    return False

In [5]:
input_mat = []
kernel_mat = []
expected_mat = []

# test case 1
input_mat.append(np.array([[1, 2, 1, 2],
                      [2, 1, 2, 1],
                      [1, 2, 1, 2],
                      [2, 1, 2, 1]]))

kernel_mat.append(np.array([[1, 0],
                       [0, 1]]))

expected_mat.append(np.array([[2, 4, 2],
                                [4, 2, 4],
                                [2, 4, 2]]))

# test case 2
input_mat.append(np.array([[1, 0, 0, 0],
                      [0, 1, 0, 0],
                      [0, 0, 1, 0],
                      [0, 0, 0, 1]]))
kernel_mat.append(np.array([[1, 0], [0, 1]]))
expected_mat.append(np.array([[2, 0, 0], [0, 2, 0], [0, 0, 2]]))


# test case 3
input_mat.append(np.array([[1, 0, 0, 0],
                      [0, 1, 0, 0],
                      [0, 0, 1, 0],
                      [0, 0, 0, 1]]))
kernel_mat.append(np.array([[1, -1],
                       [-1, 0]]))

expected_mat.append(np.array([[ 1, -1,  0], [-1,  1, -1],[ 0, -1,  1]]))


# test case 4
input_mat.append(np.array([[1, 0, 0, 0],
                      [0, 1, 0, 0],
                      [0, 0, 1, 0],
                      [0, 0, 0, 1]]))
kernel_mat.append(np.array([[1, 0, 0, 0],
                      [0, 1, 0, 0],
                      [0, 0, 1, 0],
                      [0, 0, 0, 1]]))

expected_mat.append(np.array([[4]]))


# test case 5 - should either through an error, or return empty matrix
input_mat.append(np.array([[1, -1],
                       [-1, 0]]))
kernel_mat.append(np.array([[1, 0, 0, 0],
                      [0, 1, 0, 0],
                      [0, 0, 1, 0],
                      [0, 0, 0, 1]]))

expected_mat.append([])



for i in range(len(input_mat)):
  if input_mat[i].shape[0] < kernel_mat[i].shape[0]:
    output_mat = []
  else:
    output_mat = conv2d(input_mat[i], kernel_mat[i])

  print(output_mat)
  if np.array_equal(output_mat, expected_mat[i]):
    print("Correct output!\n")
  else:
    print("Incorrect output!\n")

[[2. 4. 2.]
 [4. 2. 4.]
 [2. 4. 2.]]
Correct output!

[[2. 0. 0.]
 [0. 2. 0.]
 [0. 0. 2.]]
Correct output!

[[ 1. -1.  0.]
 [-1.  1. -1.]
 [ 0. -1.  1.]]
Correct output!

[[4.]]
Correct output!

[]
Correct output!



# Problem 2

Using only ```numpy```, implement the function ```maxpooling2d```. It takes as input ```input_mat``` and ```s``` and outputs ```output_mat```.
The variables ```input_mat``` and ```output_mat``` are square matrices and ```s``` is an integer.  It should compute the maxpooling operation 
on ```input_mat``` using window of shape ```s``` times ```s```.



# Max Pooling

Max pooling is an operation in which an input matrix and a window size of $s$ by $s$ is specified. The windows iterates throught the input matrix and considers a non overlapping region of $s$ by $s$. The maximum value in that region becomes one of the values in the output matrix. Padding is permitted if the input matrix and window size do not match.

## Padding

There are a few situations that must be handled by padding the input matrix.

### More columns than rows

Add rows of zeros until matrix is square

### More rows than columns

Add rows of columns until matrix is square

### Square input but stride length is uneven
Pad uniformly until until `dim(input_mat) % s == 0`


In [0]:
def maxpooling2d(input_mat, s):

  # handle non-square input matrix
  if input_mat.shape[0] != input_mat.shape[1]:

    # difference of number of columns and rows
    diff = input_mat.shape[0] - input_mat.shape[1]
    
    # more rows than columns
    if diff > 0:
      pad = np.zeros(input_mat.shape[1])
      input_mat = np.column_stack((input_mat, pad))

    # more columns than rows
    elif diff < 0:
      pad = np.zeros(input_mat.shape[1])
      input_mat = np.vstack((input_mat, pad))


  # pad uniformly if stride is uneven
  pad = input_mat.shape[0] % s
  input_mat = np.pad(input_mat, pad)

  # create output matrix
  n = input_mat.shape[0]
  output_dim = (int(n/s), int(n/s))
  output_mat = np.zeros(output_dim)

  # traverse input matrix in increments of s
  for i in range(0, n, s):
    for k in range(0, n, s):
      x = int(i/s)
      y = int(k/s)
      
      # output the maximum value in a subset of input matrix
      output_mat[x][y] = np.max(input_mat[i:i+s, k:k+s])

  return output_mat

In [7]:
# Test Cases

input_mat = []
expected_mat = []
s = []

input_mat.append(np.array([[1, 2, 1, 2],
                           [2, 4, 2, 1],
                           [1, 2, 4, 2],
                           [2, 1, 2, 1]]))
s.append(2)

expected_mat.append(np.array([[4, 2],
                              [2, 4]]))

input_mat.append(np.array([[1, 2, 1, 2, 4, 5],
                           [2, 4, 2, 1, 0, 3],
                           [1, 2, 4, 2, 4, 5],
                           [2, 1, 2, 1, 2, 1],
                           [1, 1, 2, 3, 1, 2]]))
s.append(2)

expected_mat.append([[4, 2, 5],
                     [2, 4, 5],
                     [1, 3, 2]])


for i in range(len(input_mat)):

  output_mat = maxpooling2d(input_mat[i], s[i])

  print(output_mat)

  if np.array_equal(output_mat, expected_mat[i]):
    print("Correct output!")
  else:
    print("Incorrect output!")
  
  print()


[[4. 2.]
 [2. 4.]]
Correct output!

[[4. 2. 5.]
 [2. 4. 5.]
 [1. 3. 2.]]
Correct output!

